# QUICK START with REDIS CLUSTER MODE


In [1]:
import redis 

In [97]:
r1 = redis.Redis(host='127.0.0.1', port=7000)
r2 = redis.Redis(host='127.0.0.1', port=7002)
r3 = redis.Redis(host='127.0.0.1', port=7004)

In [98]:
r1.set('admin', 'luis')

True

In [99]:
r1.get('admin')

b'luis'

¿Pero qué ocurre cuando intentamos insertar una clave/valor que debería ir en un servidor diferente del que estamos usando para ejecutar la instrucción SET?

In [101]:
r1.set('foo', 'bar')

ResponseError: MOVED 12182 172.18.0.4:7004

Ocurre que de nuevo no podemos conectar con la dirección DOCKER interna del tercer servidor 172.18.0.4:7004

Podemos resolverlo capturando excepciones:

In [100]:
try:
    r1.set('foo', 'bar')
    print("succeeded with node 1")
except:
    try:
        r2.set('foo', 'bar')
        print("succeeded with node 2")
    except:
        try:
            r3.set('foo','bar')
            print("succeeded with node 3")
        except:
            print("couldnt complete the set opertation")

succeeded with node 3


Pero no es muy eficiente porque tenemos que ejecutar varios comandos SET hasta que demos con el bueno.

A la hora de trabajar con un cluster de Redis casi siempre se usa el paquete redis-py-cluster. Pero en este caso parece estar afectado por los mismos problemas de conectividad de nuestro entorno windows dockerizado. Funciona pero tarda mucho tiempo.

In [16]:
from rediscluster import RedisCluster

In [17]:
startup_nodes = [{"host": "127.0.0.1", "port": "7000"}]
rc = RedisCluster(startup_nodes=startup_nodes, decode_responses=True)

In [20]:
rc.cluster_keyslot("foo")

12182

Una posible solución es averiguar cuál es el servidor sobre el que tenemos que ejecutar la instrución SET.

Podemos obtener el slot calculando manualmente el "CRC16 mod 16384" y usar ese slot para atacar directamente al servidor redis adecuado

In [48]:
import crcmod
crc16 = crcmod.mkCrcFun(0x11021, rev=False, initCrc=0x0000, xorOut=0x0000)
crc16(b'foo') % 16384

12182

In [49]:
import crcmod.predefined
xmodem_crc_func = crcmod.predefined.mkCrcFun('xmodem')
xmodem_crc_func(b'foo') % 16384

12182

Para usarlo de manera más compacta vamos a definirlo en una función

In [102]:
import crcmod
def getSlot(key):
    #key must be binary
    crc16 = crcmod.mkCrcFun(0x11021, rev=False, initCrc=0x0000, xorOut=0x0000)
    return crc16(key) % 16384

In [103]:
getSlot(b'foo')

12182

También tenemos que definir una variable PYTHON en la que tengamos HARDCODEADOS los slots de los que se encarga cada nodo. Esto podemos obtenerlo con los comandos de REDIS-CLI que devuelven información del CLUSTER:

* CLUSTER SLOTS
* CLUSTER NODES

In [104]:
SlotToServer=dict([(i,r1) for i in range(5461)]+
     [(j,r2) for j in range(5461,10923)]+
     [(k,r3) for k in range(10923,16384)])

Con esta variable que me permite hacer el mapeo SLOT --> SERVIDOR ya puedo hacer la operativa SET sin miedo a equivocarme

In [108]:
r=SlotToServer[getSlot(b'foo')]
r.set('foo','bar')

True